In [4]:
import findspark
findspark.init()

In [76]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import explode, from_json, col
from pyspark.sql.types import ArrayType, StructType, StructField, StringType
from pyspark.sql.functions import explode_outer, col
from pyspark.sql.types import StructType


In [6]:
conf = SparkConf().set("spark.network.timeout", "600s")  # Set the network timeout
spark = SparkSession.builder \
    .appName("MLops") \
    .master("spark://localhost:7077") \
    .config(conf=conf) \
    .getOrCreate()
spark = SparkSession.builder.master("spark://localhost:7077").getOrCreate()

In [7]:
spark

## Leer JSON

- Info() [json_process](https://github.com/raveendratal/ravi_azureadbadf/blob/main/azure_realtime_scenarios/dynamic_json_process.ipynb)

In [80]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode_outer, col
from pyspark.sql.types import StructType

# Definir el JSON y guardarlo en un archivo local
json_data = """
{
  "name":"MSFT",
  "location":"Redmond",
  "satellites": ["Bay Area", "Shanghai"],
  "goods": {
    "trade":true,
    "customers":["government", "distributer", "retail"],
    "orders":[
        {"orderId":1,"orderTotal":123.34,"shipped":{"orderItems":[{"itemName":"Laptop","itemQty":20},{"itemName":"Charger","itemQty":2}]}},
        {"orderId":2,"orderTotal":323.34,"shipped":{"orderItems":[{"itemName":"Mice","itemQty":2},{"itemName":"Keyboard","itemQty":1}]}}
    ]}
}
{"name":"Company1","location":"Seattle",
  "satellites": ["New York"],
  "goods":{"trade":false,
           "customers":["store1", "store2"],
           "orders":[
               {"orderId":4,"orderTotal":123.34,"shipped":{"orderItems":[{"itemName":"Laptop","itemQty":20},{"itemName":"Charger","itemQty":3}]}},
               {"orderId":5,"orderTotal":343.24,"shipped":{"orderItems":[{"itemName":"Chair","itemQty":4},{"itemName":"Lamp","itemQty":2}]}}
           ]}
}
{"name": "Company2", "location": "Bellevue",
  "goods": {"trade": true, "customers":["Bank"], "orders": [{"orderId": 4, "orderTotal": 123.34}]}}
{"name": "Company3", "location": "Kirkland"}
"""

with open("sample.json", "w") as file:
    file.write(json_data)

# Leer el JSON desde el archivo local
df_json = spark.read.option("multiLine", "true").json("sample.json")

def child_struct(nested_df):
    list_schema = [((), nested_df)]
    flat_columns = []

    while len(list_schema) > 0:
        parents, df = list_schema.pop()
        flat_cols = [col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],))) for c in df.dtypes if c[1][:6] != "struct"]
        struct_cols = [c[0] for c in df.dtypes if c[1][:6] == "struct"]
        flat_columns.extend(flat_cols)

        for i in struct_cols:
            projected_df = df.select(i + ".*")
            list_schema.append((parents + (i,), projected_df))

    return nested_df.select(flat_columns)

def master_array(df):
    array_cols = [c[0] for c in df.dtypes if c[1][:5] == "array"]
    while len(array_cols) > 0:
        for c in array_cols:
            df = df.withColumn(c, explode_outer(c))
        df = child_struct(df)
        array_cols = [c[0] for c in df.dtypes if c[1][:5] == "array"]
    return df

df_output = master_array(df_json)

df_output.show(truncate=False)


+--------+----+----------+---------------+-----------+--------------------+-----------------------+----------------------------------------+---------------------------------------+
|location|name|satellites|goods_customers|goods_trade|goods_orders_orderId|goods_orders_orderTotal|goods_orders_shipped_orderItems_itemName|goods_orders_shipped_orderItems_itemQty|
+--------+----+----------+---------------+-----------+--------------------+-----------------------+----------------------------------------+---------------------------------------+
|Redmond |MSFT|Bay Area  |government     |true       |1                   |123.34                 |Laptop                                  |20                                     |
|Redmond |MSFT|Bay Area  |government     |true       |1                   |123.34                 |Charger                                 |2                                      |
|Redmond |MSFT|Bay Area  |government     |true       |2                   |323.34              

## Leer JSON  - Anidados

In [87]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode_outer, col

# Crear la sesión de Spark
spark = SparkSession.builder.appName("example").getOrCreate()

# Definir el JSON y guardarlo en un archivo local
json_data = """
{ "user_id": "76561197970982479", "user_url": "http://steamcommunity.com/profiles/76561197970982479", "reviews": [ { "funny": "", "posted": "Posted November 5, 2011.", "last_edited": "", "item_id": "1250", "helpful": "No ratings yet", "recommend": true, "review": "Simple yet with great replayability. In my opinion does 'zombie' hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth 'zombie' splattering fun for the whole family. Amazed this sort of FPS is so rare." }, { "funny": "", "posted": "Posted July 15, 2011.", "last_edited": "", "item_id": "22200", "helpful": "No ratings yet", "recommend": true, "review": "It's unique and worth a playthrough." }, { "funny": "", "posted": "Posted April 21, 2011.", "last_edited": "", "item_id": "43110", "helpful": "No ratings yet", "recommend": true, "review": "Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!" } ] }
{ "user_id": "js41637", "user_url": "http://steamcommunity.com/id/js41637", "reviews": [ { "funny": "", "posted": "Posted June 24, 2014.", "last_edited": "", "item_id": "251610", "helpful": "15 of 20 people (75%) found this review helpful", "recommend": true, "review": 'I know what you think when you see this title "Barbie Dreamhouse Party" but do not be intimidated by it\'s title, this is easily one of my GOTYs. You don\'t get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can\'t 360 noscope your friends, but what you can do is show them up with your bad â™¥â™¥â™¥ dance moves and put them to shame as you show them what true fashion and color combinations are.I know this game says for kids but, this is easily for any age range and any age will have a blast playing this.8/8' }, { "funny": "", "posted": "Posted September 8, 2013.", "last_edited": "", "item_id": "227300", "helpful": "0 of 1 people (0%) found this review helpful", "recommend": true, "review": "For a simple (it's actually not all that simple but it can be!) truck driving Simulator, it is quite a fun and relaxing game. Playing on simple (or easy?) its just the basic WASD keys for driving but (if you want) the game can be much harder and realistic with having to manually change gears, much harder turning, etc. And reversing in this game is a â™¥â™¥â™¥â™¥â™¥, as I imagine it would be with an actual truck. Luckily, you don't have to reverse park it but you get extra points if you do cause it is bloody hard. But this is suprisingly a nice truck driving game and I had a bit of fun with it." }, { "funny": "", "posted": "Posted November 29, 2013.", "last_edited": "", "item_id": "239030", "helpful": "1 of 4 people (25%) found this review helpful", "recommend": true, "review": 'Very fun little game to play when your bored or as a time passer. Very gud. Do Recommend. pls buy' } ] }
{ "user_id": "evcentric", "user_url": "http://steamcommunity.com/id/evcentric", "reviews": [ { "funny": "", "posted": "Posted February 3.", "last_edited": "", "item_id": "248820", "helpful": "No ratings yet", "recommend": true, "review": "A suitably punishing roguelike platformer. Winning feels good. Progressive unlocks mean a good slog ending in failure doesn't feel like a waste." }, { "funny": "", "posted": "Posted December 4, 2015.", "last_edited": "Last edited December 5, 2015.", "item_id": "370360", "helpful": "No ratings yet", "recommend": true, "review": '"Run for fun? What the hell kind of fun is that?"' }, { "funny": "", "posted": "Posted November 3, 2014.", "last_edited": "", "item_id": "237930", "helpful": "No ratings yet", "recommend": true, "review": 'Elegant integration of gameplay, story, world development and aesthetic.' }, { "funny": "", "posted": "Posted October 15, 2014.", "last_edited": "", "item_id": "263360", "helpful": "No ratings yet", "recommend": true, "review": 'Random drops and random quests, with stat points. Animation style reminiscent of the era before the Voodoo card.' }, { "funny": "", "posted": "Posted October 15, 2014.", "last_edited": "", "item_id": "107200", "helpful": "No ratings yet", "recommend": true, "review": 'Fun balance of tactics and strategy. Potential for very rewarding battles on smaller maps. Can become a bit of a grind on larger maps (>200 stars).' }, { "funny": "", "posted": "Posted October 15, 2014.", "last_edited": "", "item_id": "224500", "helpful": "No ratings yet", "recommend": true, "review": 'Fun world builder, with plenty of option of how you want challenge served to you. Gnome pathing sometimes frustrating if you expand very very quickly.' } ] }
"""

with open("sample.json", "w") as file:
    file.write(json_data)

# Leer el JSON desde el archivo local
df_json = spark.read.option("multiLine", "true").json("sample.json")

# Definir funciones para manejar la estructura anidada
def flatten_reviews(df):
    return df.select("user_id", "user_url", explode_outer("reviews").alias("review"))

def flatten_review(review_df):
    return review_df.select("user_id", "user_url", col("review.*"))

# Aplicar las transformaciones
df_flattened = flatten_reviews(df_json)
df_output = flatten_review(df_flattened)

# Mostrar el resultado
df_output.show(truncate=False)



+-----------------+----------------------------------------------------+-----+--------------+-------+-----------+------------------------+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id          |user_url                                            |funny|helpful       |item_id|last_edited|posted                  |recommend|review                                                                                                                                                                                                                                                   |
+-----------------+----------------------------------------------------+-----+--------------+-------+-----------+------------------------+---------+------------------------------------------------------

### columnas con JSON parsing?

In [ ]:


# Definir el JSON y guardarlo en un archivo local
json_data = """
{ 'user_id': '76561197970982479', 'user_url': 'http://steamcommunity.com/profiles/76561197970982479', 'reviews': [ { 'funny': '', 'posted': 'Posted November 5, 2011.', 'last_edited': '', 'item_id': '1250', 'helpful': 'No ratings yet', 'recommend': true, 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.' }, { 'funny': '', 'posted': 'Posted July 15, 2011.', 'last_edited': '', 'item_id': '22200', 'helpful': 'No ratings yet', 'recommend': true, 'review': "It's unique and worth a playthrough." }, { 'funny': '', 'posted': 'Posted April 21, 2011.', 'last_edited': '', 'item_id': '43110', 'helpful': 'No ratings yet', 'recommend': true, 'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!' } ] }
{ 'user_id': 'js41637', 'user_url': 'http://steamcommunity.com/id/js41637', 'reviews': [ { 'funny': '', 'posted': 'Posted June 24, 2014.', 'last_edited': '', 'item_id': '251610', 'helpful': '15 of 20 people (75%) found this review helpful', 'recommend': true, 'review': 'I know what you think when you see this title "Barbie Dreamhouse Party" but do not be intimidated by it\'s title, this is easily one of my GOTYs. You don\'t get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can\'t 360 noscope your friends, but what you can do is show them up with your bad â™¥â™¥â™¥ dance moves and put them to shame as you show them what true fashion and color combinations are.I know this game says for kids but, this is easily for any age range and any age will have a blast playing this.8/8' }, { 'funny': '', 'posted': 'Posted September 8, 2013.', 'last_edited': '', 'item_id': '227300', 'helpful': '0 of 1 people (0%) found this review helpful', 'recommend': true, 'review': "For a simple (it's actually not all that simple but it can be!) truck driving Simulator, it is quite a fun and relaxing game. Playing on simple (or easy?) its just the basic WASD keys for driving but (if you want) the game can be much harder and realistic with having to manually change gears, much harder turning, etc. And reversing in this game is a â™¥â™¥â™¥â™¥â™¥, as I imagine it would be with an actual truck. Luckily, you don't have to reverse park it but you get extra points if you do cause it is bloody hard. But this is suprisingly a nice truck driving game and I had a bit of fun with it." }, { 'funny': '', 'posted': 'Posted November 29, 2013.', 'last_edited': '', 'item_id': '239030', 'helpful': '1 of 4 people (25%) found this review helpful', 'recommend': true, 'review': 'Very fun little game to play when your bored or as a time passer. Very gud. Do Recommend. pls buy' } ] }
{ 'user_id': 'evcentric', 'user_url': 'http://steamcommunity.com/id/evcentric', 'reviews': [ { 'funny': '', 'posted': 'Posted February 3.', 'last_edited': '', 'item_id': '248820', 'helpful': 'No ratings yet', 'recommend': true, 'review': "A suitably punishing roguelike platformer. Winning feels good. Progressive unlocks mean a good slog ending in failure doesn't feel like a waste." }, { 'funny': '', 'posted': 'Posted December 4, 2015.', 'last_edited': 'Last edited December 5, 2015.', 'item_id': '370360', 'helpful': 'No ratings yet', 'recommend': true, 'review': '"Run for fun? What the hell kind of fun is that?"' }, { 'funny': '', 'posted': 'Posted November 3, 2014.', 'last_edited': '', 'item_id': '237930', 'helpful': 'No ratings yet', 'recommend': true, 'review': 'Elegant integration of gameplay, story, world development and aesthetic.' }, { 'funny': '', 'posted': 'Posted October 15, 2014.', 'last_edited': '', 'item_id': '263360', 'helpful': 'No ratings yet', 'recommend': true, 'review': 'Random drops and random quests, with stat points. Animation style reminiscent of the era before the Voodoo card.' }, { 'funny': '', 'posted': 'Posted October 15, 2014.', 'last_edited': '', 'item_id': '107200', 'helpful': 'No ratings yet', 'recommend': true, 'review': 'Fun balance of tactics and strategy. Potential for very rewarding battles on smaller maps. Can become a bit of a grind on larger maps (>200 stars).' }, { 'funny': '', 'posted': 'Posted October 15, 2014.', 'last_edited': '', 'item_id': '224500', 'helpful': 'No ratings yet', 'recommend': true, 'review': 'Fun world builder, with plenty of option of how you want challenge served to you. Gnome pathing sometimes frustrating if you expand very very quickly.' } ] }
"""

with open("sample.json", "w") as file:
    file.write(json_data)

# Leer el JSON desde el archivo local
df_json = spark.read.option("multiLine", "true").json("sample.json")

# Definir funciones para manejar la estructura anidada
def flatten_reviews(df):
    return df.select("user_id", "user_url", explode_outer("reviews").alias("review"))

def flatten_review(review_df):
    return review_df.select("user_id", "user_url", col("review.*"))

# Aplicar las transformaciones
df_flattened = flatten_reviews(df_json)
df_output = flatten_review(df_flattened)

# Mostrar el resultado
df_output.show(truncate=False)


## TRABAJO

In [ ]:
import json
df = spark.read.json("../datasets/raw/australian_user_reviews.json")
#df = spark.read.json(r"C:\\Users\\ozi\ti\\pry_ml_ops\\datasets\\raw\\australian_user_reviews.json")
# Ruta al archivo
file_path = "../datasets/raw/australian_user_reviews.json"

# Leer el contenido del archivo con codificación utf-8
with open(file_path, 'r', encoding='utf-8') as file:
    json_data = file.read()

# Imprimir el contenido del JSON para identificar el problema
print(json_data)

In [70]:
from pyspark.sql.functions import size, col, coalesce

# Definir el esquema detallado para "reviews"
review_schema = StructType([
    StructField("funny", StringType(), True),
    StructField("posted", StringType(), True),
    StructField("last_edited", StringType(), True),
    StructField("item_id", StringType(), True),
    StructField("helpful", StringType(), True),
    StructField("recommend", BooleanType(), True),
    StructField("review", StringType(), True),
])

# Definir el esquema principal
main_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("user_url", StringType(), True),
    StructField("reviews", ArrayType(review_schema), True),
])

# Cargar los datos en un DataFrame aplicando el esquema
df = spark.read.json("../datasets/raw/australian_user_reviews.json", schema=main_schema)

# Reemplazar valores nulos en la columna 'reviews' con un array vacío
df_fixed_reviews = df.withColumn('reviews', coalesce('reviews', array()))

# Agregar una nueva columna 'num_reviews' que contiene el tamaño del array 'reviews'
df_with_num_reviews_fixed = df_fixed_reviews.withColumn('num_reviews', size('reviews'))

# Mostrar el resultado
df_with_num_reviews_fixed.show(truncate=False)


+-----------------+----------------------------------------------------+-------+-----------+
|user_id          |user_url                                            |reviews|num_reviews|
+-----------------+----------------------------------------------------+-------+-----------+
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479|[]     |0          |
|js41637          |http://steamcommunity.com/id/js41637                |[]     |0          |
|evcentric        |http://steamcommunity.com/id/evcentric              |[]     |0          |
|doctr            |http://steamcommunity.com/id/doctr                  |[]     |0          |
|maplemage        |http://steamcommunity.com/id/maplemage              |[]     |0          |
|Wackky           |http://steamcommunity.com/id/Wackky                 |[]     |0          |
|76561198079601835|http://steamcommunity.com/profiles/76561198079601835|[]     |0          |
|MeaTCompany      |http://steamcommunity.com/id/MeaTCompany           

In [71]:
# Contar nulos por columna
for column in df.columns:
    print(f"Columna '{column}': {df.filter(col(column).isNull()).count()} nulos")

Columna 'user_id': 0 nulos
Columna 'user_url': 0 nulos
Columna 'reviews': 25771 nulos


In [72]:
print((df.count(), len(df.columns)))

(25799, 3)


In [75]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, ArrayType 

# Definir el esquema para cada review
review_schema = StructType([
    StructField("funny", StringType(), True),
    StructField("posted", StringType(), True),
    StructField("last_edited", StringType(), True),
    StructField("item_id", StringType(), True),
    StructField("helpful", StringType(), True),  
    StructField("recommend", BooleanType(), True),
    StructField("review", StringType(), True)
])

# Definir el esquema principal con el array de reviews
main_schema = StructType([
  StructField("user_id", StringType(), True),
  StructField("user_url", StringType(), True),
  StructField("reviews", ArrayType(review_schema), True)  
])

# Leer el JSON aplicando el esquema
df = spark.read.schema(main_schema).json("../datasets/raw/australian_user_reviews.json")

# Verificar que se leyó correctamente
df.printSchema()  
df.show(truncate=False)

root
 |-- user_id: string (nullable = true)
 |-- user_url: string (nullable = true)
 |-- reviews: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- funny: string (nullable = true)
 |    |    |-- posted: string (nullable = true)
 |    |    |-- last_edited: string (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- helpful: string (nullable = true)
 |    |    |-- recommend: boolean (nullable = true)
 |    |    |-- review: string (nullable = true)

+-----------------+----------------------------------------------------+-------+
|user_id          |user_url                                            |reviews|
+-----------------+----------------------------------------------------+-------+
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479|NULL   |
|js41637          |http://steamcommunity.com/id/js41637                |NULL   |
|evcentric        |http://steamcommunity.com/id/evcentric              |NULL  

## basic por soliconar 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, BooleanType
from pyspark.sql.functions import from_json, col

# Create a Spark session
spark = SparkSession.builder.appName("LecturaJSON").getOrCreate()

# Load the DataFrame
df = spark.read.text("../datasets/raw/steam_reviews.json.gz")

# Define the schema for the JSON content
json_schema = StructType([
    StructField("username", StringType(), True),
    StructField("hours", DoubleType(), True),
    StructField("products", IntegerType(), True),
    StructField("product_id", StringType(), True),
    StructField("page_order", IntegerType(), True),
    StructField("date", StringType(), True),
    StructField("text", StringType(), True),
    StructField("early_access", BooleanType(), True),
    StructField("page", IntegerType(), True)
])

# Apply the schema using the from_json function directly to the value column
df = df.withColumn("data", from_json(col("value"), json_schema))

# Select the columns from the nested structure
reviews_df = df.select("data.*")



df.show(5, truncate=False)
# Show the schema and some rows
reviews_df.printSchema()
reviews_df.show(5, truncate=False)


## Leer de froma basica con caracteres

In [ ]:
def clean_text(text):
    # Reemplazar caracteres no ASCII con un espacio en blanco
    cleaned_text = ''.join(char if ord(char) < 128 else ' ' for char in text)
    return cleaned_text

def read_and_clean_reviews(file_path):
    cleaned_reviews = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # Limpiar caracteres no ASCII
                cleaned_line = clean_text(line)
                
                # Convertir la línea limpiada a un diccionario
                review_dict = eval(cleaned_line)
                
                # Agregar el diccionario limpiado a la lista de revisiones
                cleaned_reviews.append(review_dict)
            except Exception as e:
                print(f"Error al procesar la línea: {line}")
                print(f"Error: {e}")
    
    return cleaned_reviews

# Ejemplo de uso
file_path = 'australian_user_reviews.json'  # Reemplaza 'ruta_del_archivo.txt' con la ruta real de tu archivo
reviews = read_and_clean_reviews(file_path)

# Ahora 'reviews' es una lista de diccionarios con las revisiones limpias
for review in reviews:
    print(review)


## Leer un json a pyspark

In [ ]:

def clean_text(text):
    # Reemplazar caracteres no ASCII con un espacio en blanco
    cleaned_text = ''.join(char if ord(char) < 128 else ' ' for char in text)
    return cleaned_text

def read_and_clean_reviews(file_path):
    cleaned_reviews = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # Limpiar caracteres no ASCII
                cleaned_line = clean_text(line)
                
                # Convertir la línea limpiada a un diccionario
                review_dict = eval(cleaned_line)
                
                # Agregar el diccionario limpiado a la lista de revisiones
                cleaned_reviews.append(review_dict)
            except Exception as e:
                print(f"Error al procesar la línea: {line}")
                print(f"Error: {e}")
    
    return cleaned_reviews

# Ejemplo de uso
file_path = 'australian_user_reviews.json'  # Reemplaza 'ruta_del_archivo.txt' con la ruta real de tu archivo
reviews = read_and_clean_reviews(file_path)

# Crear una lista de tuplas con la información necesaria
data = []
for review in reviews:
    user_id = review['user_id']
    user_url = review['user_url']
    user_reviews = review.get('reviews', [])
    
    for user_review in user_reviews:
        item_id = user_review.get('item_id', '')
        recommend = user_review.get('recommend', False)
        review_text = user_review.get('review', '')
        funny = user_review.get('funny', '')
        posted = user_review.get('posted', '')
        last_edited = user_review.get('last_edited', '')
        helpful = user_review.get('helpful', '')

        data.append((user_id, user_url, item_id, recommend, review_text, funny, posted, last_edited, helpful))

# Crear el esquema del DataFrame
schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("user_url", StringType(), True),
    StructField("item_id", StringType(), True),
    StructField("recommend", BooleanType(), True),
    StructField("review", StringType(), True),
    StructField("funny", StringType(), True),
    StructField("posted", StringType(), True),
    StructField("last_edited", StringType(), True),
    StructField("helpful", StringType(), True)
])

# Crear el DataFrame de PySpark
df = spark.createDataFrame(data, schema=schema)

# Mostrar el DataFrame
df.show(truncate=False)

# Detener la sesión de Spark
#spark.stop()
